In [1]:
login_creds = {
    'user_id' : "",
    'password' : "!",
    "twofa_value" : '743691'
}

In [2]:
import pyotp, requests 
session = requests.Session() 

res1 = session.post(
    'https://kite.zerodha.com/api/login',
    data={
        "user_id": login_creds['user_id'], 
        "password": login_creds['password'], 
        "type": "user_id"}
) 

loginRes = res1.json() 
#loginRes

In [3]:
finalRes = session.post(
    'https://kite.zerodha.com/api/twofa',
    data={
        "user_id": login_creds['user_id'], 
        "request_id": loginRes['data']['request_id'], 
        "twofa_value": login_creds['twofa_value'],
        "twofa_type": 'app_code'
    }
) 

finalRes.json() 

{'status': 'success', 'data': {'profile': {}}}

In [4]:
session_info = session.cookies.get_dict()

headers = {
    "cookie": f"kf_session={session_info['kf_session']}; __cf_bm={session_info['__cf_bm']}; _cfuvid={session_info['_cfuvid']}",
    "Authorization": f"enctoken {session_info['enctoken']}"
}

In [5]:
# holdings

holdings = requests.request("GET",
                             "https://kite.zerodha.com/oms/portfolio/holdings", 
                             headers=headers)
holdings.json()

{'status': 'success',
 'data': [{'tradingsymbol': '1003VCL25',
   'exchange': 'BSE',
   'instrument_token': 240313860,
   'isin': 'INE01HV07437',
   'product': 'CNC',
   'price': 0,
   'quantity': 9,
   'used_quantity': 0,
   't1_quantity': 0,
   'realised_quantity': 9,
   'authorised_quantity': 0,
   'authorised_date': '2024-10-01 00:00:00',
   'authorisation': {},
   'opening_quantity': 9,
   'short_quantity': 0,
   'collateral_quantity': 0,
   'collateral_type': '',
   'discrepancy': True,
   'average_price': 0,
   'last_price': 994,
   'close_price': 994,
   'pnl': 8946,
   'day_change': 0,
   'day_change_percentage': 0},
  {'tradingsymbol': '945IFSL25',
   'exchange': 'BSE',
   'instrument_token': 240227332,
   'isin': 'INE321N07301',
   'product': 'CNC',
   'price': 0,
   'quantity': 13,
   'used_quantity': 0,
   't1_quantity': 0,
   'realised_quantity': 13,
   'authorised_quantity': 0,
   'authorised_date': '2024-10-01 00:00:00',
   'authorisation': {},
   'opening_quantity': 13

In [6]:
# positions

positions = requests.request("GET",
                 "https://kite.zerodha.com/oms/portfolio/positions", 
                 headers=headers)
positions.json()

{'status': 'success',
 'data': {'net': [{'tradingsymbol': 'ADANIENSOL',
    'exchange': 'NSE',
    'instrument_token': 2615553,
    'product': 'CNC',
    'quantity': 10,
    'overnight_quantity': 0,
    'multiplier': 1,
    'average_price': 1028.505,
    'close_price': 0,
    'last_price': 1040.1,
    'value': -10285.050000000001,
    'pnl': 115.94999999999891,
    'm2m': 115.94999999999891,
    'unrealised': 115.94999999999891,
    'realised': 0,
    'buy_quantity': 10,
    'buy_price': 1028.505,
    'buy_value': 10285.050000000001,
    'buy_m2m': 10285.050000000001,
    'sell_quantity': 0,
    'sell_price': 0,
    'sell_value': 0,
    'sell_m2m': 0,
    'day_buy_quantity': 10,
    'day_buy_price': 1028.505,
    'day_buy_value': 10285.050000000001,
    'day_sell_quantity': 0,
    'day_sell_price': 0,
    'day_sell_value': 0},
   {'tradingsymbol': 'VBL',
    'exchange': 'NSE',
    'instrument_token': 4843777,
    'product': 'CNC',
    'quantity': 8,
    'overnight_quantity': 0,
    'mu

In [7]:
# orders

orders = requests.request("GET",
                 "https://kite.zerodha.com/oms/orders", 
                 headers=headers)
orders.json()

{'status': 'success',
 'data': [{'account_id': 'QOZ373',
   'placed_by': 'QOZ373',
   'order_id': '241001000103252',
   'exchange_order_id': '1000000000684442',
   'parent_order_id': None,
   'status': 'COMPLETE',
   'status_message': None,
   'status_message_raw': None,
   'order_timestamp': '2024-10-01 09:16:24',
   'exchange_update_timestamp': '2024-10-01 09:16:24',
   'exchange_timestamp': '2024-10-01 09:16:24',
   'variety': 'regular',
   'modified': False,
   'exchange': 'NSE',
   'tradingsymbol': 'ADANIENSOL',
   'instrument_token': 2615553,
   'order_type': 'MARKET',
   'transaction_type': 'BUY',
   'validity': 'DAY',
   'validity_ttl': 0,
   'product': 'CNC',
   'quantity': 10,
   'disclosed_quantity': 0,
   'price': 0,
   'trigger_price': 0,
   'average_price': 1028.505,
   'filled_quantity': 10,
   'pending_quantity': 0,
   'cancelled_quantity': 0,
   'market_protection': 0,
   'meta': {},
   'tag': None,
   'guid': '19Xtoaqyvsbulns'},
  {'account_id': 'QOZ373',
   'placed_b

In [8]:
# margins

margins = requests.request("GET",
                 "https://kite.zerodha.com/oms/user/margins", 
                 headers=headers)
margins.json()

{'status': 'success',
 'data': {'equity': {'enabled': True,
   'net': 61952.2,
   'available': {'adhoc_margin': 0,
    'cash': 73065.4,
    'opening_balance': 73065.4,
    'live_balance': 61952.2,
    'collateral': 0,
    'intraday_payin': 15080},
   'utilised': {'debits': 26193.2,
    'exposure': 0,
    'm2m_realised': 0,
    'm2m_unrealised': 0,
    'option_premium': 0,
    'payout': 0,
    'span': 0,
    'holding_sales': 0,
    'turnover': 0,
    'liquid_collateral': 0,
    'stock_collateral': 0,
    'equity': 0,
    'delivery': 0}},
  'commodity': {'enabled': False,
   'net': 0,
   'available': {'adhoc_margin': 0,
    'cash': 0,
    'opening_balance': 0,
    'live_balance': 0,
    'collateral': 0,
    'intraday_payin': 0},
   'utilised': {'debits': 0,
    'exposure': 0,
    'm2m_realised': 0,
    'm2m_unrealised': 0,
    'option_premium': 0,
    'payout': 0,
    'span': 0,
    'holding_sales': 0,
    'turnover': 0,
    'liquid_collateral': 0,
    'stock_collateral': 0,
    'equity'